# Neural Network

神经网络（Neural Network）是一种模拟生物神经系统的计算模型，广泛应用于机器学习和人工智能领域。神经网络由多个神经元（Neuron）组成，这些神经元通过连接（Connection）形成网络结构。以下是神经网络的基本概念和原理。

### 神经网络的基本概念

1. **神经元（Neuron）**：
   - 神经元是神经网络的基本单位，类似于生物神经元。
   - 每个神经元接收多个输入信号，通过加权求和和激活函数处理后，输出一个信号。

2. **权重（Weight）**：
   - 权重是连接神经元的参数，表示输入信号的重要性。
   - 权重在训练过程中不断调整，以最小化损失函数。

3. **偏置（Bias）**：
   - 偏置是一个额外的参数，用于调整神经元的输出。
   - 偏置在训练过程中也会不断调整。

4. **激活函数（Activation Function）**：
   - 激活函数用于引入非线性，使神经网络能够处理复杂的非线性问题。
   - 常见的激活函数包括 Sigmoid、ReLU（Rectified Linear Unit）、Tanh 等。

5. **层（Layer）**：
   - 神经网络由多个层组成，每层包含多个神经元。
   - 输入层（Input Layer）：接收输入数据。
   - 隐藏层（Hidden Layer）：处理输入数据，提取特征。
   - 输出层（Output Layer）：输出预测结果。

### 神经网络的工作原理

#### 1.前向传播 (Forward Propagation)
在前向传播过程中，输入数据通过网络的各层传递，最终生成输出结果

1. **输入数据传递到输入层**：
   - 输入数据（例如，MNIST 图像展平后的向量）作为输入层的输入。

2. **输入层到隐藏层的传递**：
   - 输入层的输出（即输入数据）传递到第一个隐藏层。
   - 每个隐藏层的神经元计算输入信号的加权和，并通过激活函数处理后输出信号。

3. **隐藏层到输出层的传递**：
   - 隐藏层的输出传递到输出层。
   - 输出层的神经元计算输入信号的加权和，并通过激活函数处理后输出信号。

4. **输出层的输出**：
   - 输出层的输出即为神经网络的预测结果。

##### 数学表示
假设我们有一个简单的两层神经网络，输入层有 $ n $ 个神经元，隐藏层有 $ h $ 个神经元，输出层有 $ m $ 个神经元。
1. **输入层到隐藏层**：
   - 输入数据： $ \mathbf{x} \in \mathbb{R}^n $
   - 权重矩阵： $ \mathbf{W}_1 \in \mathbb{R}^{h \times n} $
   - 偏置向量： $ \mathbf{b}_1 \in \mathbb{R}^h $
   - 激活函数： $ \sigma(\cdot) , \cdot \in \mathbb{R}^h $
   - 隐藏层输出： $ \mathbf{h} = \sigma(\mathbf{W}_1 \mathbf{x} + \mathbf{b}_1) $

2. **隐藏层到输出层**：
   - 权重矩阵： $ \mathbf{W}_2 \in \mathbb{R}^{m \times h} $
   - 偏置向量： $ \mathbf{b}_2 \in \mathbb{R}^m $
   - 激活函数： $ \sigma(\cdot) , \cdot \in \mathbb{R}^m $
   - 输出层输出： $ \mathbf{y} = \sigma(\mathbf{W}_2 \mathbf{h} + \mathbf{b}_2) $

##### 具体实现
以下是使用 PyTorch 实现前向传播的示例代码。

###### 定义神经网络模型
```python
import torch
import torch.nn as nn

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # 输入层到隐藏层的全连接层
        self.relu = nn.ReLU()  # 激活函数
        self.fc2 = nn.Linear(hidden_size, output_size)  # 隐藏层到输出层的全连接层

    def forward(self, x):
        out = self.fc1(x)  # 输入层到隐藏层
        out = self.relu(out)  # 激活函数
        out = self.fc2(out)  # 隐藏层到输出层
        return out
```

###### 前向传播示例
```python
# 超参数
input_size = 784  # 输入层大小（28x28 的图像展平为向量）
hidden_size = 256  # 隐藏层大小
output_size = 10  # 输出层大小（10 个类别）

# 初始化模型
model = SimpleNN(input_size, hidden_size, output_size)

# 示例输入数据（一个批次的 MNIST 图像展平后的向量）
input_data = torch.randn(64, input_size)  # 64 是批次大小

# 前向传播
output = model(input_data)

print(output.shape)  # 输出的形状应为 (64, 10)
```

##### 解释
1. **定义神经网络模型**：
   - `self.fc1 = nn.Linear(input_size, hidden_size)`：定义输入层到隐藏层的全连接层。
   - `self.relu = nn.ReLU()`：定义激活函数。
   - `self.fc2 = nn.Linear(hidden_size, output_size)`：定义隐藏层到输出层的全连接层。

2. **前向传播**：
   - `out = self.fc1(x)`：输入数据通过输入层传递到隐藏层，计算加权和。
   - `out = self.relu(out)`：隐藏层的输出通过激活函数处理。
   - `out = self.fc2(out)`：隐藏层的输出传递到输出层，计算加权和。

3. **示例输入数据**：
   - `input_data = torch.randn(64, input_size)`：生成一个批次的随机输入数据，形状为 (64, 784)。
   - `output = model(input_data)`：进行前向传播，计算输出结果。


#### 2.损失函数 (Loss Function)
损失函数用于衡量神经网络的预测结果与真实值之间的差异。它是训练过程中优化的目标，神经网络通过最小化损失函数来调整其参数。
##### 常见的损失函数
1. **均方误差（Mean Squared Error, MSE）**：
   - 主要用于回归问题。
   - 公式：$$ \text{MSE}(y,\hat{y}) = \frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y}_i)^2 \quad , \quad y,\hat{y} \in \mathbb{R}^N $$
   - 其中，$ y_i $ 是真实值，$ \hat{y}_i $ 是预测值，$ N $ 是样本数量。

2. **交叉熵损失（Cross-Entropy Loss）**：
   - 主要用于分类问题。
   - 公式：$$ \text{Cross-Entropy Loss} = -\frac{1}{N} \sum_{i=1}^{N} \sum_{c=1}^{C} y_{i,c} \log(\hat{y}_{i,c}) $$
   - 其中，$ y_{i,c} $ 是真实标签的 one-hot 编码，$ \hat{y}_{i,c} $ 是预测概率，$ C $ 是类别数量。

##### example

###### 1. MSE

假设我们有一个简单的回归问题，真实值和预测值如下：
- 真实值 $ y = [2.0, 3.0, 4.0] $
- 预测值 $ \hat{y} = [2.5, 2.8, 4.2] $
- 计算均方误差：
   $$
   \text{MSE} = \frac{1}{3} [(2.0 - 2.5)^2 + (3.0 - 2.8)^2 + (4.0 - 4.2)^2] = \frac{0.33}{3} = 0.11
   $$

###### 2. Binary-Cross-Entropy Loss
- C = 2 时的 Cross-Entropy Loss, 用于2分类问题


交叉熵损失主要用于分类问题，用于衡量预测概率分布与真实标签分布之间的差异。

#### 例子
假设我们有一个简单的分类问题，真实标签和预测概率如下：
- 真实标签 $ y = [1, 0, 0] $（表示类别 0）
- 预测概率 $ \hat{y} = [0.7, 0.2, 0.1] $

#### 计算过程
1. 计算交叉熵损失：
   $$
   \text{Cross-Entropy Loss} = - \sum_{c=1}^{3} y_c \log(\hat{y}_c)
   $$
   其中，$ y_c $ 是真实标签的 one-hot 编码，$ \hat{y}_c $ 是预测概率。

2. 具体计算：
   $$
   \text{Cross-Entropy Loss} = - (1 \cdot \log(0.7) + 0 \cdot \log(0.2) + 0 \cdot \log(0.1)) = - \log(0.7) \approx 0.357
   $$

### 代码示例
以下是使用 PyTorch 实现均方误差和交叉熵损失的代码示例。

#### 均方误差（MSE）示例
```python
import torch
import torch.nn as nn

# 真实值和预测值
y_true = torch.tensor([2.0, 3.0, 4.0])
y_pred = torch.tensor([2.5, 2.8, 4.2])

# 定义均方误差损失函数
mse_loss = nn.MSELoss()

# 计算损失
loss = mse_loss(y_pred, y_true)
print(f'MSE Loss: {loss.item():.4f}')
```

#### 交叉熵损失（Cross-Entropy Loss）示例
```python
import torch
import torch.nn as nn

# 真实标签和预测概率
y_true = torch.tensor([0])  # 类别 0
y_pred = torch.tensor([[0.7, 0.2, 0.1]])  # 预测概率

# 定义交叉熵损失函数
cross_entropy_loss = nn.CrossEntropyLoss()

# 计算损失
loss = cross_entropy_loss(y_pred, y_true)
print(f'Cross-Entropy Loss: {loss.item():.4f}')
```

### 解释
1. **均方误差（MSE）**：
   - 计算每个样本的误差平方，并取平均值。
   - 示例代码中，使用 PyTorch 的 `nn.MSELoss` 计算均方误差。

2. **交叉熵损失（Cross-Entropy Loss）**：
   - 计算预测概率分布与真实标签分布之间的差异。
   - 示例代码中，使用 PyTorch 的 `nn.CrossEntropyLoss` 计算交叉熵损失。

通过这种方式，你可以理解均方误差和交叉熵损失的计算过程和具体实现。如果你有任何问题，请随时提问。


### 3. 反向传播（Backward Propagation）
反向传播用于计算损失函数相对于每个参数的梯度。通过链式法则，梯度从输出层逐层传递到输入层。

#### 反向传播的步骤
1. **计算输出层的梯度**：
   - 计算损失函数相对于输出层的梯度。

2. **计算隐藏层的梯度**：
   - 使用链式法则，将输出层的梯度传递到隐藏层，计算损失函数相对于隐藏层的梯度。

3. **计算输入层的梯度**：
   - 使用链式法则，将隐藏层的梯度传递到输入层，计算损失函数相对于输入层的梯度。

### 4. 梯度下降（Gradient Descent）
梯度下降用于更新神经网络的参数，以最小化损失函数。常见的梯度下降算法包括批量梯度下降（Batch Gradient Descent）、随机梯度下降（Stochastic Gradient Descent, SGD）和小批量梯度下降（Mini-Batch Gradient Descent）。

#### 梯度下降的步骤
1. **计算梯度**：
   - 使用反向传播计算损失函数相对于每个参数的梯度。

2. **更新参数**：
   - 使用梯度下降算法更新参数。
   - 公式：\[ \theta = \theta - \eta \nabla_\theta J(\theta) \]
   - 其中，\( \theta \) 是参数，\( \eta \) 是学习率，\( \nabla_\theta J(\theta) \) 是损失函数相对于参数的梯度。

### 具体实现
以下是使用 PyTorch 实现损失函数、反向传播和梯度下降的示例代码。

#### 定义神经网络模型
```python
import torch
import torch.nn as nn
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # 输入层到隐藏层的全连接层
        self.relu = nn.ReLU()  # 激活函数
        self.fc2 = nn.Linear(hidden_size, output_size)  # 隐藏层到输出层的全连接层

    def forward(self, x):
        out = self.fc1(x)  # 输入层到隐藏层
        out = self.relu(out)  # 激活函数
        out = self.fc2(out)  # 隐藏层到输出层
        return out
```

#### 训练神经网络
```python
# 超参数
input_size = 784  # 输入层大小（28x28 的图像展平为向量）
hidden_size = 256  # 隐藏层大小
output_size = 10  # 输出层大小（10 个类别）
learning_rate = 0.001
epochs = 10

# 初始化模型、损失函数和优化器
model = SimpleNN(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()  # 交叉熵损失
optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Adam 优化器

# 加载数据
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.MNIST(root='../data', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

# 训练神经网络
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 28*28)  # 将图像展平为向量

        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()  # 梯度清零
        loss.backward()  # 反向传播: 计算梯度
        optimizer.step()  # 更新参数

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
```

### 解释
1. **定义神经网络模型**：
   - `self.fc1 = nn.Linear(input_size, hidden_size)`：定义输入层到隐藏层的全连接层。
   - `self.relu = nn.ReLU()`：定义激活函数。
   - `self.fc2 = nn.Linear(hidden_size, output_size)`：定义隐藏层到输出层的全连接层。

2. **初始化模型、损失函数和优化器**：
   - `criterion = nn.CrossEntropyLoss()`：定义交叉熵损失函数。
   - `optimizer = optim.Adam(model.parameters(), lr=learning_rate)`：定义 Adam 优化器。

3. **前向传播**：
   - `outputs = model(images)`：输入数据通过网络的各层传递，生成输出结果。

4. **计算损失**：
   - `loss = criterion(outputs, labels)`：计算预测结果与真实值之间的差异。

5. **反向传播和优化**：
   - `optimizer.zero_grad()`：梯度清零，确保每次计算梯度时不会累加上一次计算的梯度。
   - `loss.backward()`：反向传播，计算损失函数相对于每个参数的梯度。
   - `optimizer.step()`：使用梯度下降算法更新参数。

通过这种方式，你可以理解神经网络训练过程中的损失函数、反向传播和梯度下降的具体实现和发生的过程。如果你有任何问题，请随时提问。

3. **反向传播（Backward Propagation）**：
   - 反向传播用于计算损失函数相对于每个权重和偏置的梯度。
   - 通过链式法则，梯度从输出层逐层传递到输入层。

4. **梯度下降（[Gradient Descent](./Gradient_Descent.ipynb)）**：
   - 梯度下降用于更新权重和偏置，以最小化损失函数。
   - 常见的梯度下降算法包括批量梯度下降（Batch Gradient Descent）、随机梯度下降（Stochastic Gradient Descent, SGD）和小批量梯度下降（Mini-Batch Gradient Descent）。

### 神经网络的训练过程

1. **初始化**：
   - 初始化神经网络的权重和偏置。

2. **前向传播**：
   - 输入数据通过神经网络，计算输出结果。

3. **计算损失**：
   - 使用损失函数计算预测结果与真实值之间的差异。

4. **反向传播**：
   - 计算损失函数相对于每个权重和偏置的梯度。

5. **更新参数**：
   - 使用梯度下降算法更新权重和偏置。

6. **重复**：
   - 重复前向传播、计算损失、反向传播和更新参数的过程，直到损失函数收敛或达到预定的训练轮数。

## 代码示例: 用神经网络识别 MNIST 手写数字 的类别(标签, 0-9)

对于 [MNIST 数据集](./MNIST_dataset.ipynb#MNIST-dataset-shape) 的单个图像的形状是 [1,28,28],展平为向量后形状为 [784], 作为神经网络的输入.
$$
\begin{bmatrix}
x_0 \\
x_1  \\
x_2  \\
\vdots \\
x_{783}  \\
\end{bmatrix}
$$
输入层: 784个输入神经元, 隐藏层: 建议选择:一层,256个神经元, 输出层: 10个神经元, 分别对应0-9的数字.

In [5]:
# 1. 导入必要的库
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# 2. 定义数据预处理和加载数据
# 定义数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图像转换为张量
    transforms.Normalize((0.5,), (0.5,))  # 归一化到 [-1, 1]
])

# 加载训练集和测试集
train_dataset = datasets.MNIST(root='../data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='../data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# 3. 定义神经网络模型
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# 超参数
input_size = 784  # 输入层大小（28x28 的图像展平为向量）
hidden_size = 256  # 隐藏层大小
output_size = 10  # 输出层大小（10 个类别）
learning_rate = 0.001
epochs = 10
# 4. 初始化模型、损失函数和优化器
model = SimpleNN(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# 5. 训练神经网络
for epoch in range(epochs):
    # 内层循环使用 train_loader 加载数据, 进行小批量的数据读取
    for i, (images, labels) in enumerate(train_loader):
        # if i == 3: # 为了快速验证代码的正确性，只迭代3次
        #     break
        # 内层每次迭代时，都会进行一次 梯度下降算法, 包括5个步骤
        # 将图像展平为向量
        # print(f"batch_idx: {i}, images.shape: {images.shape}, labels.shape: {labels.shape}")
        # print(f"labels: {labels}")
        images = images.view(-1, 28*28) # .shape = (64, 784)
        # print(images.shape) 
        
        outputs = model(images) # 1.前向传播: 计算输出
        loss = criterion(outputs, labels) # 2.计算输出和标签之间的损失

        # 这三步的顺序
        optimizer.zero_grad() # 3.梯度清零: 以确保每次计算梯度时不会累加上一次计算的梯度
        loss.backward() # 4.反向传播: 计算梯度
        optimizer.step() # 5.更新参数

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
# 保存模型
simple_nn_model_dir = '../output/weights/simple_nn_model/'
torch.save(model.state_dict(), simple_nn_model_dir + 'simple_nn_model.pth')
# 6. 评估神经网络
# 评估模型在测试集上的性能
model.eval()  # 设置模型为评估模式
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.view(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the 10000 test images: {100 * correct / total:.2f}%')

Epoch [1/10], Step [100/938], Loss: 0.4736
Epoch [1/10], Step [200/938], Loss: 0.5091
Epoch [1/10], Step [300/938], Loss: 0.3434
Epoch [1/10], Step [400/938], Loss: 0.1720
Epoch [1/10], Step [500/938], Loss: 0.2657
Epoch [1/10], Step [600/938], Loss: 0.2019
Epoch [1/10], Step [700/938], Loss: 0.3240
Epoch [1/10], Step [800/938], Loss: 0.1952
Epoch [1/10], Step [900/938], Loss: 0.2109
Epoch [2/10], Step [100/938], Loss: 0.1176
Epoch [2/10], Step [200/938], Loss: 0.1964
Epoch [2/10], Step [300/938], Loss: 0.0366
Epoch [2/10], Step [400/938], Loss: 0.1276
Epoch [2/10], Step [500/938], Loss: 0.0917
Epoch [2/10], Step [600/938], Loss: 0.0600
Epoch [2/10], Step [700/938], Loss: 0.0919
Epoch [2/10], Step [800/938], Loss: 0.0819
Epoch [2/10], Step [900/938], Loss: 0.1962
Epoch [3/10], Step [100/938], Loss: 0.0840
Epoch [3/10], Step [200/938], Loss: 0.0656
Epoch [3/10], Step [300/938], Loss: 0.0716
Epoch [3/10], Step [400/938], Loss: 0.0981
Epoch [3/10], Step [500/938], Loss: 0.1767
Epoch [3/10

RuntimeError: Parent directory ../output/weights/simple_nn_model does not exist.